In [20]:
from fyers_apiv3.FyersWebsocket import data_ws
import datetime
import os

symbol_script = 'NSE:NIFTYBANK-INDEX'

current_dir1 = os.path.dirname(os.getcwd())
current_dir2 = os.path.dirname(current_dir1)
file_path = os.path.join(current_dir2, 'access.txt')

def get_access_token(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    return file_content

def onmessage(data):
    print(data)
    script = data['symbol']
    ltp = data['ltp']
    high = data['high_price']
    low = data['low_price']
    ltt= datetime.datetime.fromtimestamp( data['exch_feed_time'] )
    #print (f"Script: {script}, Ltp:{ltp}, High: {high}, Low: {low}, ltt:{ltt}")
    current_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=5, minutes=30)))

    if current_time.hour == 17 and current_time.minute == 25:
        print("Unsubscribing at 3:25 PM IST")
        data_type = "SymbolUpdate"
        symbols_to_unsubscribe = [symbol_script]
        fyers.unsubscribe(symbols=symbols_to_unsubscribe, data_type=data_type)


def onerror(err):
    print("Error:", err)


def onclose(msg):
    print("Connection closed:", msg)


def onopen():
    data_type = "SymbolUpdate"
    symbols = [symbol_script]
    fyers.subscribe(symbols=symbols, data_type=data_type)
    fyers.keep_running()

access_token = get_access_token(file_path)

fyers = data_ws.FyersDataSocket(
    access_token=access_token,       
    log_path="",                     
    litemode=False,                  
    write_to_file=False,              
    reconnect=True,                  
    on_connect=onopen,               
    on_close=onclose,                
    on_error=onerror,                
    on_message=onmessage             
)

data = fyers.connect()
print(data)


{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error: KeyError: The key 'symbol' is missing in the response.
{'type': 'ful', 'code': 200, 'message': 'Full Mode On', 's': 'ok'}
Error: KeyError: The key 'symbol' is missing in the response.
None


{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error: KeyError: The key 'symbol' is missing in the response.
{'ltp': 47295.6, 'prev_close_price': 47242.65, 'ch': 52.95, 'chp': 0.11, 'exch_feed_time': 1704870303, 'high_price': 47386.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47290.55, 'prev_close_price': 47242.65, 'ch': 47.9, 'chp': 0.1, 'exch_feed_time': 1704870304, 'high_price': 47386.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47296.9, 'prev_close_price': 47242.65, 'ch': 54.25, 'chp': 0.11, 'exch_feed_time': 1704870305, 'high_price': 47386.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47295.2, 'prev_close_price': 47242.65, 'ch': 52.55, 'chp': 0.11, 'exch_feed_time': 1704870306, 'high_price': 47386.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-IN

In [6]:
from fyers_apiv3.FyersWebsocket import data_ws
from datetime import datetime, timedelta
import os

current_dir1 = os.path.dirname(os.getcwd())
current_dir2 = os.path.dirname(current_dir1)
file_path = os.path.join(current_dir2, 'access.txt')

def get_access_token(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    return file_content

def update_candle(current_candle, data, time_frame_minutes):
    timestamp = datetime.utcfromtimestamp(data.get("exch_feed_time", 0))
    if current_candle is None:
        current_candle = {
            "timestamp": timestamp,
            "open": data["ltp"],
            "high": data["ltp"],
            "low": data["ltp"],
            "close": data["ltp"],
        }
    else:
        if timestamp - current_candle["timestamp"] >= timedelta(minutes=time_frame_minutes):
            current_candle["close"] = data["ltp"]
            # Reset current candle
            current_candle = {
                "timestamp": timestamp,
                "open": data["ltp"],
                "high": data["ltp"],
                "low": data["ltp"],
                "close": data["ltp"],
            }
        else:
            # Update current candle
            current_candle["close"] = data["ltp"]
            current_candle["high"] = max(current_candle["high"], data["ltp"])
            current_candle["low"] = min(current_candle["low"], data["ltp"])

    return current_candle


def on_message(message, current_candle, time_frame_minutes):
    current_candle = update_candle(current_candle, message, time_frame_minutes)
    print('current Candle',current_candle)

def on_error(message):
    print("Error:", message)

def on_close(message):
    print("Connection closed:", message)

def on_open(fyers_instance, time_frame_minutes):
    data_type = "SymbolUpdate"
    symbols = ['BSE:BANKEX-INDEX']
    fyers_instance.subscribe(symbols=symbols, data_type=data_type)
    fyers_instance.keep_running()

def connect_to_websocket(access_token, time_frame_minutes):
    fyers_instance = data_ws.FyersDataSocket(
        access_token=access_token,
        log_path="",
        litemode=False,
        write_to_file=False,
        reconnect=True,
        on_connect=lambda: on_open(fyers_instance, time_frame_minutes),
        on_close=on_close,
        on_error=on_error,
        on_message=lambda msg: on_message(msg, current_candle, time_frame_minutes)
    )

    current_candle = None
    fyers_instance.connect()

# Example usage
access_token = get_access_token(file_path)
time_frame_minutes = 5

connect_to_websocket(access_token, time_frame_minutes)


Error: KeyError: The key 'ltp' is missing in the response.
Error: KeyError: The key 'ltp' is missing in the response.


Error: KeyError: The key 'ltp' is missing in the response.
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 50), 'open': 53336.61, 'high': 53336.61, 'low': 53336.61, 'close': 53336.61}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 51), 'open': 53336.61, 'high': 53336.61, 'low': 53336.61, 'close': 53336.61}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 52), 'open': 53336.81, 'high': 53336.81, 'low': 53336.81, 'close': 53336.81}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 52), 'open': 53336.81, 'high': 53336.81, 'low': 53336.81, 'close': 53336.81}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 52), 'open': 53336.81, 'high': 53336.81, 'low': 53336.81, 'close': 53336.81}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 53), 'open': 53335.02, 'high': 53335.02, 'low': 53335.02, 'close': 53335.02}
current Candle {'timestamp': datetime.datetime(2024, 1, 10, 4, 59, 54), 'open

In [21]:
from fyers_apiv3 import fyersModel


client_id='6MS9YXZMMR-100'
symbol_script = 'NSE:NIFTYBANK-INDEX'

current_dir1 = os.path.dirname(os.getcwd())
current_dir2 = os.path.dirname(current_dir1)
file_path = os.path.join(current_dir2, 'access.txt')

def get_access_token(file_path):
    with open(file_path, "r") as file:
        file_content = file.read()
    return file_content

access_token = get_access_token(file_path)

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol":symbol_script,
    "resolution":"D",
    "date_format":"0",
    "range_from":"1690895316",
    "range_to":"1691068173",
    "cont_flag":"1"
}

response = fyers.history(data=data)
print(response)



{
  "candles":[
      [
        1690934400,
        609.85,
        610.5,
        594.1,
        598.45,
        14977497
      ],
      [
        1691020800,
        598.7,
        600.85,
        585,
        590.5,
        27774877
      ]
  ],
  "code":200,
  "message":"",
  "s":"ok"
}


{'ltp': 47327.9, 'prev_close_price': 47242.65, 'ch': 85.25, 'chp': 0.18, 'exch_feed_time': 1704879476, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47328.95, 'prev_close_price': 47242.65, 'ch': 86.3, 'chp': 0.18, 'exch_feed_time': 1704879477, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47324.5, 'prev_close_price': 47242.65, 'ch': 81.85, 'chp': 0.17, 'exch_feed_time': 1704879478, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47330.4, 'prev_close_price': 47242.65, 'ch': 87.75, 'chp': 0.19, 'exch_feed_time': 1704879479, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47327.1, 'prev_close_price': 47242.65, 'ch': 84.45, 'chp': 0.18, 'exch_feed_time': 1704879480, 'high_price': 474

{'candles': [[1690934400, 609.85, 610.5, 594.1, 598.45, 14977497],
  [1691020800, 598.7, 600.85, 585, 590.5, 27774877]],
 'code': 200,
 'message': '',
 's': 'ok'}

{'ltp': 47331.15, 'prev_close_price': 47242.65, 'ch': 88.5, 'chp': 0.19, 'exch_feed_time': 1704879481, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47331.55, 'prev_close_price': 47242.65, 'ch': 88.9, 'chp': 0.19, 'exch_feed_time': 1704879482, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47338.15, 'prev_close_price': 47242.65, 'ch': 95.5, 'chp': 0.2, 'exch_feed_time': 1704879483, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47331.7, 'prev_close_price': 47242.65, 'ch': 89.05, 'chp': 0.19, 'exch_feed_time': 1704879484, 'high_price': 47423.45, 'low_price': 47010.8, 'open_price': 47073.85, 'type': 'if', 'symbol': 'NSE:NIFTYBANK-INDEX'}
{'ltp': 47336.95, 'prev_close_price': 47242.65, 'ch': 94.3, 'chp': 0.2, 'exch_feed_time': 1704879485, 'high_price': 47423